Installing Libraries

In [ ]:
!pip install segmentation-models-pytorch --quiet


In [ ]:
!pip install pytorch-forecasting


Building Baseline U-Net model for segmentation

Loading Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

base_dir = "/content/drive/MyDrive/"
orig_dir = os.path.join(base_dir, "processed_npy")   # original images & masks
aug_dir  = os.path.join(base_dir, "Hackathonn")      # augmented images & masks


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index_to_color = {
    0: (0, 255, 255),     # Urban land
    1: (255, 255, 0),     # Agriculture land
    2: (255, 0, 255),     # Rangeland
    3: (0, 255, 0),       # Forest land
    4: (0, 0, 255),       # Water
    5: (255, 255, 255),   # Barren land
    6: (0, 0, 0)          # Unknown
}

def label_mask_to_rgb(mask):
    rgb_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for idx, color in index_to_color.items():
        rgb_mask[mask == idx] = color
    return rgb_mask


In [ ]:
def visualize_npy_sample(image_path, mask_path):
    img = np.load(image_path)
    mask = np.load(mask_path)
    rgb_mask = label_mask_to_rgb(mask)

    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(img)
    axs[0].set_title("Image")
    axs[1].imshow(rgb_mask)
    axs[1].set_title("Mask")
    plt.show()


Custom Pytorch Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class SegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths  = mask_paths
        self.transform   = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = np.load(self.image_paths[idx]).astype(np.float32)  # HWC
        msk = np.load(self.mask_paths[idx]).astype(np.uint8)     # HW

        if self.transform:
            img, msk = self.transform(img, msk)

        img_tensor = torch.from_numpy(img).permute(2, 0, 1)       # CHW
        msk_tensor = torch.from_numpy(msk).long()
        return img_tensor, msk_tensor


In [ ]:
from torch.utils.data import random_split, DataLoader, ConcatDataset

# Original dataset
orig_bases = [f.replace("_image.npy", "") for f in os.listdir(orig_dir) if f.endswith("_image.npy")]
orig_imgs = [os.path.join(orig_dir, f"{b}_image.npy") for b in orig_bases]
orig_msks = [os.path.join(orig_dir, f"{b}_mask.npy")  for b in orig_bases]
orig_dataset = SegmentationDataset(orig_imgs, orig_msks)

# 85% train / 15% test
n_train = int(0.85 * len(orig_dataset))
train_orig, test_dataset = random_split(orig_dataset, [n_train, len(orig_dataset)-n_train], generator=torch.Generator().manual_seed(42))

# Augmented dataset
aug_bases = [f.replace("_image.npy", "") for f in os.listdir(aug_dir) if f.endswith("_image.npy")]
aug_imgs = [os.path.join(aug_dir, f"{b}_image.npy") for b in aug_bases]
aug_msks = [os.path.join(aug_dir, f"{b}_mask.npy") for b in aug_bases]
aug_dataset = SegmentationDataset(aug_imgs, aug_msks)

# Combine
train_dataset = ConcatDataset([train_orig, aug_dataset])

# Dataloaders
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2)


Defining Seal Loss

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SEALLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, gamma=0.75, class_frequencies=None, eps=1e-6):
        super(SEALLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.eps = eps

        if class_frequencies is not None:
            class_frequencies = torch.tensor(class_frequencies, dtype=torch.float32)
            self.class_weights = 1.0 / (class_frequencies + eps)
            self.class_weights = self.class_weights / self.class_weights.sum()
        else:
            self.class_weights = None

    def compute_entropy(self, probs):
        log_probs = torch.clamp(probs, min=self.eps).log()
        entropy = -torch.sum(probs * log_probs, dim=1, keepdim=True)
        entropy = entropy / torch.log(torch.tensor(probs.size(1), dtype=torch.float32))
        return entropy

    def focal_tversky_loss(self, probs, targets):
        TP = (probs * targets).sum(dim=(2, 3))
        FP = (probs * (1 - targets)).sum(dim=(2, 3))
        FN = ((1 - probs) * targets).sum(dim=(2, 3))
        tversky = (TP + self.eps) / (TP + self.alpha * FP + self.beta * FN + self.eps)
        return torch.pow(1.0 - tversky, self.gamma)

    def class_balanced_iou_loss(self, probs, targets):
        intersection = (probs * targets).sum(dim=(2, 3))
        union = probs.sum(dim=(2, 3)) + targets.sum(dim=(2, 3)) - intersection
        iou = (intersection + self.eps) / (union + self.eps)
        loss = 1.0 - iou
        if self.class_weights is not None:
            class_weights = self.class_weights.to(loss.device)
            loss = loss * class_weights.unsqueeze(0)
        return loss

    def forward(self, logits, targets):
        if logits.shape != targets.shape:
            targets = F.one_hot(targets.long(), num_classes=logits.shape[1])
            targets = targets.permute(0, 3, 1, 2).float()

        probs = F.softmax(logits, dim=1)
        entropy = self.compute_entropy(probs).expand_as(probs)
        ftl = self.focal_tversky_loss(probs, targets)
        cb_iou = self.class_balanced_iou_loss(probs, targets)
        entropy_mean = (entropy * targets).sum(dim=(2, 3)) / (targets.sum(dim=(2, 3)) + self.eps)
        seal_loss = (1 - entropy_mean) * ftl + entropy_mean * cb_iou
        return seal_loss.mean()


Building Model

In [ ]:
!pip install ranger21

from ranger21 import Ranger21
import segmentation_models_pytorch as smp
from torchmetrics import JaccardIndex, F1Score
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load your model
model = smp.Unet("resnet34", encoder_weights="imagenet", in_channels=3, classes=7).to(device)

# Your custom SEAL loss
class_freqs =[0.1074, 0.5766, 0.0842, 0.1140, 0.0314, 0.0839, 0.0022]
loss_fn = SEALLoss(class_frequencies=class_freqs).to(device)

# Optimizer
optimizer = Ranger21(
    model.parameters(),
    lr=1e-4,
    num_epochs=10,
    num_batches_per_epoch=len(train_loader)
)

# Evaluation Metrics
train_iou = JaccardIndex(task="multiclass", num_classes=7, average="macro").to(device)
test_iou  = JaccardIndex(task="multiclass", num_classes=7, average="macro").to(device)
train_f1  = F1Score(task="multiclass", num_classes=7, average="macro").to(device)
test_f1   = F1Score(task="multiclass", num_classes=7, average="macro").to(device)
per_class_iou = JaccardIndex(task="multiclass", num_classes=7, average=None).to(device)


Device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Ranger21 optimizer ready with following settings:

Core optimizer = AdamW
Learning rate of 0.0001

Important - num_epochs of training = ** 10 epochs **
please confirm this is correct or warmup and warmdown will be off

Warm-up: linear warmup, over 2000 iterations

Lookahead active, merging every 5 steps, with blend factor of 0.5
Norm Loss active, factor = 0.0001
Stable weight decay of 0.0001
Gradient Centralization = On

Adaptive Gradient Clipping = True
	clipping value of 0.01
	steps for clipping = 0.001

Warm-down: Linear warmdown, starting at 72.0%, iteration 5889 of 8180
warm down will decay until 3e-05 lr


Defining Training and Testing Loop

In [ ]:
epochs=10

from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_iou.reset(); train_f1.reset()

    for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [train]"):
        X, y = X.to(device), y.to(device)

        logits = model(X)
        loss = loss_fn(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X.size(0)
        preds = torch.argmax(logits, dim=1)
        train_iou.update(preds, y)
        train_f1.update(preds, y)

    train_loss /= len(train_loader.dataset)
    mean_iou = train_iou.compute().item()
    f1 = train_f1.compute().item()

    model.eval()
    test_loss = 0.0
    test_iou.reset(); test_f1.reset()

    with torch.inference_mode():
        for X, y in tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [test]"):
            X, y = X.to(device), y.to(device)
            logits = model(X)
            loss = loss_fn(logits, y)

            test_loss += loss.item() * X.size(0)
            preds = torch.argmax(logits, dim=1)
            test_iou.update(preds, y)
            test_f1.update(preds, y)

    test_loss /= len(test_loader.dataset)
    val_iou = test_iou.compute().item()
    val_f1 = test_f1.compute().item()
    class_iou = per_class_iou(preds, y).tolist()

    print(f"\n[Epoch {epoch+1}]")
    print(f"Train Loss: {train_loss:.4f} | Train mIoU: {mean_iou:.4f} | Train F1: {f1:.4f}")
    print(f" Val  Loss: {test_loss:.4f} |  Val  mIoU: {val_iou:.4f} |  Val  F1: {val_f1:.4f}")
    print(f"Class-wise IoU: {class_iou}")

    torch.cuda.empty_cache()


Epoch 1/10 [train]:   0%|          | 0/818 [00:00<?, ?it/s]

params size saved
total param groups = 1
total params in groups = 140


Epoch 1/10 [test]: 100%|██████████| 121/121 [03:06<00:00,  1.54s/it]



[Epoch 1]
Train Loss: 0.4957 | Train mIoU: 0.0418 | Train F1: 0.0789
 Val  Loss: 0.4759 |  Val  mIoU: 0.0394 |  Val  F1: 0.0744
Class-wise IoU: [0.061410218477249146, 0.017274903133511543, 0.4928533136844635, 0.0, 0.011071565560996532, 0.1333390176296234, 0.0]


Epoch 2/10 [test]: 100%|██████████| 121/121 [03:13<00:00,  1.60s/it]


...

Epoch 6/10 [test]: 100%|██████████| 121/121 [03:04<00:00,  1.52s/it]



[Epoch 6]
Train Loss: 0.4736 | Train mIoU: 0.1113 | Train F1: 0.1650
 Val  Loss: 0.4707 |  Val  mIoU: 0.1043 |  Val  F1: 0.1536
Class-wise IoU: [0.001629198668524623, 0.0674050897359848, 0.5923591256141663, 0.0, 0.001139810192398727, 0.0016238020034506917, 0.00012259033974260092]


Epoch 7/10 [test]: 100%|██████████| 121/121 [03:04<00:00,  1.52s/it]



[Epoch 7]
Train Loss: 0.4735 | Train mIoU: 0.1269 | Train F1: 0.1937
 Val  Loss: 0.4707 |  Val  mIoU: 0.1054 |  Val  F1: 0.1528
Class-wise IoU: [0.004218069836497307, 0.012083563953638077, 0.5444685816764832, 0.0, 0.0003346022276673466, 0.006770881358534098, 6.690082227578387e-05]


Epoch 8/10 [train]:  18%|█▊        | 144/818 [03:57<18:22,  1.64s/it]